In [95]:
import pandas as pd
import numpy as np
import gurobipy as grb

In [96]:
#reading the data
data = pd.read_excel('small_data.xlsx', sheetname = None)

In [97]:
data.keys()

odict_keys(['Fulfilment Centers', 'Regions', 'Distances', 'Items', 'Demand'])

In [98]:
fc = data['Fulfilment Centers']
regions = data['Regions']
dist = data['Distances']
items = data['Items']
demand = data['Demand']

In [99]:
fc_dict=dict(zip(fc['FC_name'],fc.capacity))

In [100]:
wt_dict = dict(zip(items['item_ID'],items['shipping_weight']))

In [101]:
st_dict = dict(zip(items.item_ID, items.storage_size))

In [44]:
regions

,region_ID,region_name,population_millions,lat,long
0,0,"Kings County, NY",16.133722,41.547035,-74.786319
1,1,"Santa Clara County, CA",15.146029,37.572332,-121.399244
2,2,"Miami-Dade County, FL",14.543862,27.626951,-81.558710


In [126]:
mod = grb.Model()
#decision variable
x= {}
for name in fc['FC_name']:
    for region in regions['region_ID']:
        for item in items['item_ID']:
            x[name,region,item]=mod.addVar(lb=0,name='x[{0},{1},{2}]'.format(name,region,item))

In [127]:
#constraints
capacity_constr = {}
for name in fc_dict.keys():
    capacity_constr[name] = mod.addConstr(sum(x[name, region, item]*st_dict[item] for region in regions['region_ID'] for item in st_dict.keys() ) <=fc_dict[name], name = 'Capacity for FC '+name)

demand_constr = {}
for region in regions['region_ID']:
    for item in items['item_ID']:
        demand_constr[region]=mod.addConstr(sum(x[name,region,item] for name in fc['FC_name'])>=demand[region][item], name='demand for region '+ str(region) + ' and item '+str(item))

In [128]:
#objective
mod.setObjective(1.38*sum(x[name,region, item]*dist[name][region]*wt_dict[item] for name in fc['FC_name'] for region in regions['region_ID'] for item in wt_dict.keys()))

In [129]:
mod.setParam('OutputFlag',False)
mod.optimize()

In [154]:
print("Optimal Value is {0}".format(mod.ObjVal))

Optimal Value is 3400.769189999999


In [150]:
print('Optimal solution')
optSol = {}
for var in mod.getVars():
    if(var.x!=0):
        optSol[var.VarName]= var.x
        print('\t{0}={1}'.format(var.VarName,var.x))

Optimal solution
	x[A,0,1]=125.0
	x[A,1,0]=100.0
	x[A,1,1]=200.0
	x[A,2,1]=100.0
	x[B,0,0]=500.0
	x[B,0,1]=75.0
	x[B,2,0]=350.0


In [144]:
print("Shadow Price")
##We only care about shadow price of capacity as we do not control demand. 
shadow = {}
for constr in mod.getConstrs():
    print('\t{0}: {1}'.format(constr.ConstrName, constr.PI))
    shadow[constr.ConstrName]=constr.PI

Shadow Price
	Capacity for FC A: 0.0
	Capacity for FC B: -0.6362076
	demand for region 0 and item 0: 3.2425859999999993
	demand for region 0 and item 1: 2.141208
	demand for region 1 and item 0: 5.956631999999999
	demand for region 1 and item 1: 1.985544
	demand for region 2 and item 0: 2.4390534
	demand for region 2 and item 1: 1.4100149999999998


In [148]:
SP = pd.DataFrame.from_dict(shadow, orient='index')

In [151]:
OS = pd.DataFrame.from_dict(optSol, orient='index')

In [158]:
SP.to_excel(writer, sheet_name='Shadow Price')
OS.to_excel(writer, sheet_name='Optimal Solution')

In [165]:
df =pd.DataFrame([mod.ObjVal])

In [166]:
df.to_excel(writer, sheet_name='Optimal Value')

In [167]:
writer.save()